## K-nearest neighbors with dynamic time warping

In [ ]:
pip install tsfresh tslearn

In [ ]:
from tsfresh.examples import load_robot_execution_failures
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures
download_robot_execution_failures()
df_ts, y = load_robot_execution_failures()


In [ ]:
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
extracted_features = impute(extract_features(df_ts, column_id="id", column_sort="time"))
features_filtered = select_features(extracted_features, y)


In [ ]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
knn = KNeighborsTimeSeriesClassifier()
param_search = {
    'metric' : ['dtw'],
    'n_neighbors': [1, 2, 3]
}
tscv = TimeSeriesSplit(n_splits=2)
gsearch = GridSearchCV(
    estimator=knn,
    cv=tscv,
    param_grid=param_search
)
gsearch.fit(
    features_filtered,
    y
)


## Shapelets

In [ ]:
from sktime.classification.shapelet_based import ShapeletTransformClassifier
from sktime.classification.sklearn import RotationForest
from sktime.datasets import load_unit_test
X_train, y_train = load_unit_test(split="train", return_X_y=True)
X_test, y_test = load_unit_test(split="test", return_X_y=True)
clf = ShapeletTransformClassifier(
    estimator=RotationForest(n_estimators=3),
    n_shapelet_samples=100,
    max_shapelets=10,
    batch_size=20,
)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


## ROCKET

In [ ]:
from sktime.datasets import load_basic_motions
X_train, y_train = load_basic_motions(split="train", return_X_y=True)


In [ ]:
from sktime.transformations.panel.rocket import Rocket
rocket = Rocket()
rocket.fit(X_train)
X_train_transform = rocket.transform(X_train)


In [ ]:
from sklearn.linear_model import RidgeClassifierCV
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True)
classifier.fit(X_train_transform, y_train)


In [ ]:
X_test_transform = rocket.transform(X_test)

In [ ]:
classifier.score(X_test_transform, y_test)

## ProximityForest

In [ ]:
from sktime.classification.distance_based import ProximityForest
from sktime.datasets import load_unit_test
X_train, y_train = load_unit_test(split="train", return_X_y=True)
X_test, y_test = load_unit_test(split="test", return_X_y=True)
clf = ProximityForest(n_estimators=5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


## Mr-SEQL 

In [ ]:
from sklearn.model_selection import train_test_split
from sktime.datasets import load_arrow_head
X, y = load_arrow_head(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [ ]:
from sktime.classification.shapelet_based import MrSEQLClassifier
ms = MrSEQLClassifier(seql_mode="clf")
ms.fit(X_train, y_train)


In [ ]:
from sklearn import metrics
predicted = ms.predict(X_test)
print("Accuracy with mr-seql: %2.3f" % metrics.accuracy_score(y_test, predicted))


## HIVE-COTE

In [ ]:
from sktime.classification.hybrid import HIVECOTEV1
from sktime.contrib.vector_classifiers._rotation_forest import RotationForest
from sktime.datasets import load_unit_test
X_train, y_train = load_unit_test(split="train", return_X_y=True)
X_test, y_test = load_unit_test(split="test", return_X_y=True)
clf = HIVECOTEV1(
    stc_params={
        "estimator": RotationForest(n_estimators=3),
        "n_shapelet_samples": 500,
        "max_shapelets": 20, 
        "batch_size": 100, 
    },
    tsf_params={"n_estimators": 10},
    rise_params={"n_estimators": 10},
    cboss_params={"n_parameter_samples": 25, "max_ensemble_size": 5},
)
clf.fit(X_train, y_train) HIVECOTEV1(...) y_pred = clf.predict(X_test)
